In [1]:
from newspaper import Article
from transformers import AutoConfig, AutoTokenizer, AutoModel
from summarizer import Summarizer
import pandas as pd
from transformers import PegasusForConditionalGeneration, AutoTokenizer
import torch
from newspaper import Config
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

import torch


/opt/bitnami/miniconda/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/opt/bitnami/miniconda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Testing the newspaper module
## (According to Prof.Nwala one of the best boilerplate removers for articles)

In [2]:


url = "https://www.npr.org/2022/11/12/1136205315/musk-twitter-bankruptcy-how-likely"
article = Article(url)

# Need to download then parse article to get the text.

article.download()
article.parse()

print(article.text)

Elon Musk says Twitter bankruptcy is possible, but is that likely?

Enlarge this image toggle caption Jeff Chiu/AP Jeff Chiu/AP

Before Elon Musk took over Twitter, it was hardly a gangbusters business. The company is only occasionally profitable. Its userbase and advertising revenue is puny compared to social media rivals like Facebook and TikTok.

Yet the possibility of a looming Twitter bankruptcy was not a fate anyone ever seriously discussed.

Now, however, under the chaotic leadership of Musk, the mercurial billionaire has reportedly told staff that bankruptcy could be nigh if Twitter does not start making more money.

How could this be the case, and what exactly has changed?

In taking the company private in his $44 billion purchase of Twitter, Musk cashed in some of his Tesla stock and also saddled the social platform with $13 billion in debt, which is a massive obligation for a company the size of Twitter.

For context, the debt equals about seven times the company's projected

In [3]:
## Pretty good! No junk.

## Running a test run with BERT and Pegasus

In [5]:

def generate_summary_from_text(text):

    model = Summarizer()
    result = model(text, ratio=0.2)

    return result

BERT_Summary = generate_summary_from_text(article.text)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
src_text = article.text

## CODE USAGE PULLED FROM: https://huggingface.co/docs/transformers/model_doc/pegasus#usage-example
## https://huggingface.co/google/pegasus-large contains a list of all different models to use.

In [7]:

model_name = 'google/pegasus-multi_news'
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

/opt/bitnami/jupyterhub-singleuser/.local/lib/python3.7/site-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 256 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


## BERT Model vs. Pegasus Model

## Producing Sample Pegasus Summary Database from GDELT URLs with Failure Mitigation

In [8]:
df = pd.read_csv("gdelt.csv")
urls = df["URLs"]

In [9]:
# Will store final results here : URL : Summary is the format.
results = {}

In [ ]:


for item in urls:
    
    ## fixes browser block ... 
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    ## prevents timing out ...
    config.request_timeout = 30
    try:
        ## Get article text.
        article = Article(url, config=config)
        article.download()
        article.parse()
        txt = article.text
        # Multi_news has performed well so far.
        model_name = 'google/pegasus-multi_news'
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
        batch = tokenizer(txt, truncation=True, padding='longest', return_tensors="pt").to(device)
        translated = model.generate(**batch)
        tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
        results[url] = tgt_text
    except Exception as e:
        print(e)



In [ ]:
## Clean out printing junk....

for key, value in results.items():
    
    results_clean[key] = (value[0].replace("", "\n"))



In [ ]:
pegasus_df = pd.DataFrame(columns=["URL", "Summary"])
pegasus_df["URL"] = results_clean.keys()
pegasus_df["Summary"] = results_clean.values()
pegasus_df.to_csv("pegasus_test.csv")

pegasus_df.head()